In [2]:
import joblib

exp = joblib.load("../../../tests/assets/expel/expel_experiences_10_fake.joblib")

In [4]:
experiences = []
for i in exp:
    experiences.append({
        "question": i.question,
        "key": i.key,
        "trajectory": i.trajectory,
        "reflections": i.reflections
    })

In [7]:
joblib.dump(experiences, "expel_experiences_10_fake_new.joblib")

['expel_experiences_10_fake_new.joblib']